# Telechargement de données auto

In [9]:
from snappy import ProductIO
from snappy import jpy
from snappy import GPF

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

import zipfile
import os 

In [2]:
# connect to the API
# user copernicus
#user = 'jonathancourtois'
# user eumetsat
user = 'jonathanc'
password = 'Tutorat2019'
#api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')
api = SentinelAPI(user, password, 'https://coda.eumetsat.int')

# download single scene by known product id
#api.download(<product_id>)

In [3]:
# search by polygon, time, and Hub query keywords
#footprint = geojson_to_wkt(read_geojson('polygonRhone.geojson'))
footprint = 'POLYGON ((3.632678257283426 44.218520813901634, 5.593941712716586 43.91153565143112,5.122306465193042 42.40131912218008, 3.208397257788463 42.706154058381806,3.632678257283426 44.218520813901634))'
products = api.query(footprint,
                     date = ('20191001','NOW'),
                     platformname = 'Sentinel-3',
                     area_relation='Intersects', 
                     producttype='OL_2_WFR___',
                     #timeliness='Non Time Critical',
                     instrumentshortname = 'OLCI',
                     onlinequalitycheck = 'PASSED',
                     productlevel = 'L2')
                     
                     #cloudcoverpercentage = (0, 30)) # only for copernicus
print(len(products))

Querying products: 100%|██████████| 105/105 [00:00<00:00, 253.74 products/s]


In [4]:
print(len(products))
print(products)

105
OrderedDict([(u'4e212a56-6839-40ce-8ca7-0d980e804740', {u'salinewatercoverpercentage': 10.0, u'beginposition': datetime.datetime(2019, 11, 1, 9, 50, 37, 350000), u'coastalcoverpercentage': 0.006433, u'onlinequalitycheck': u'PASSED', u'processinglevel': u'2', u'orbitdirection': u'descending', u'orbitnumber': 19302, u'producttype': u'OL_2_WFR___', u'passdirection': u'descending', u'productlevel': u'L2', u'platformname': u'Sentinel-3', u'size': u'185.52 MB', u'sensoroperationalmode': u'Earth Observation', u'instrumentshortname': u'OLCI', u'ecmwf': u'ANALYSIS', u'timeliness': u'Non Time Critical', u'platformidentifier': u'2016-011A', u'filename': u'S3A_OL_2_WFR____20191101T095037_20191101T095337_20191122T060545_0180_051_079_2160_MAR_O_NT_002.SEN3', u'link_alternative': u"http://coda.eumetsat.int/odata/v1/Products('4e212a56-6839-40ce-8ca7-0d980e804740')/", u'gmlfootprint': u'<gml:Polygon srsName="http://www.opengis.net/gml/srs/epsg.xml#4326" xmlns:gml="http://www.opengis.net/gml">\n   <

In [5]:
# download all results from the search
#a = api.download_all(products,directory_path='home/courtois/jupyter-sentinel/images')
#print(a)

In [6]:
# convert to Pandas DataFrame
products_df = api.to_dataframe(products)
print(products_df)

                                      salinewatercoverpercentage  \
026533bd-f4b9-49e5-b814-27c0be4831f3                         8.0   
034ad03d-7443-41a6-adda-8ddc113e1965                        11.0   
054099d9-a9c0-47b1-90b2-18a682c63a3b                        17.0   
07436289-a78d-4b62-aedf-c77797ad9669                         3.0   
0ed6a473-98a1-44f9-bbfb-91c76f6458d5                        15.0   
0fa44694-1dce-4d85-94db-b259945bdd31                         9.0   
11257fa4-1247-43f5-8429-defa037e53d8                         9.0   
1accc22b-3c0c-4d2f-81c7-90de21119140                         9.0   
1c6924b4-34e0-46a5-b971-d10e08bb273b                        16.0   
1c9194e0-6ddf-4d59-91c1-e012dc5c9f02                         7.0   
216b9690-30bb-4da0-a2c2-c2691fb02bdb                        16.0   
2431f912-c507-4167-a7ea-f4219174a4ff                        10.0   
2ca3e930-7f61-4356-a094-920eb2407a2c                        14.0   
33092cad-d2cb-4956-8d6d-c5f9cc724e4f            

In [7]:
# sort and limit to first 5 sorted products
products_df_sorted = products_df.sort_values(['salinewatercoverpercentage', 'ingestiondate'], ascending=[False, True])
products_df_sorted = products_df_sorted.head(2)
print(products_df_sorted.index)

Index([u'4b62117e-e8c3-434d-9c28-e6374c1dbce9', u'c6761fa3-beaa-40c1-a6a4-0f6214c72999'], dtype='object')


In [17]:
# download sorted and reduced products
api.download_all(products_df_sorted.index)

Downloading: 100%|██████████| 305M/305M [00:24<00:00, 12.4MB/s] 
Downloading: 100%|██████████| 281M/281M [00:22<00:00, 12.6MB/s] 


(OrderedDict([(u'4b62117e-e8c3-434d-9c28-e6374c1dbce9',
               {'date': datetime.datetime(2019, 10, 6, 10, 38, 38, 55000),
                'downloaded_bytes': 304848409,
                'footprint': 'POLYGON((-11.3288 41.9905,-10.5125 41.9213,-9.68951 41.8391,-8.86343 41.7505,-8.0294 41.6548,-7.21661 41.5555,-6.39731 41.4493,-5.57175 41.3351,-4.76556 41.2209,-3.9547 41.0973,-3.16235 40.9641,-2.36275 40.8303,-1.56427 40.6906,-0.758204 40.5445,0.029493 40.3946,0.817176 40.2387,1.58956 40.0817,2.37049 39.915,3.14825 39.7455,3.92152 39.5676,4.9698 42.1655,6.10293 44.7584,7.33398 47.3402,8.68158 49.9084,7.76675 50.1153,6.8481 50.3106,5.92381 50.501,5.00265 50.6801,4.06341 50.8556,3.11587 51.0238,2.14736 51.1855,1.18712 51.338,0.2232 51.4822,-0.737376 51.6238,-1.71466 51.7524,-2.70055 51.8706,-3.69936 51.985,-4.69421 52.0889,-5.69475 52.1846,-6.70011 52.2718,-7.70814 52.3505,-8.71268 52.4201,-9.71173 52.4744,-10.1074 49.8548,-10.509 47.2331,-10.9163 44.6107,-11.3288 41.9905))',
     

We need to unzip the files :

In [19]:
zip_file_path = "/home/courtois/jupyter-sentinel"
file_list = os.listdir("/home/courtois/jupyter-sentinel/")
# select the directory where the files are and where you want to unzip it
abs_path = []
# list for the .zip file to unzip
c=0
for a in file_list:
    if a.endswith(".zip"):  # select only the .zip files in the folder
        x = zip_file_path+'/'+a
        c += 1
        print x
        abs_path.append(x)  # put the .zip file in the list
for f in abs_path:
    zip=zipfile.ZipFile(f)
    zip.extractall(zip_file_path)  # unzip the file in the zip_file_path
    os.remove(f)   # delet the .zip file

('/home/courtois/jupyter-sentinel/S3B_OL_2_WFR____20191006T102550_20191006T102850_20191007T190005_0179_030_336_2160_MAR_O_NT_002.zip file number:', 1)
('/home/courtois/jupyter-sentinel/S3B_OL_2_WFR____20191121T103322_20191121T103622_20191121T123442_0179_032_222_2160_MAR_O_NR_002.zip file number:', 2)


La fonction Download_olci_data(...) permet de télecharger des images selon certain critères.

In [1]:
def Download_olci_data(user,password,url,footprint,datestart,dateend,zip_file_path,image_number):
    from snappy import ProductIO
    from snappy import jpy
    from snappy import GPF

    import numpy as np

    from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
    from datetime import date

    import zipfile
    import os 
    #user = 'jonathanc'
    #password = 'Tutorat2019'
    #api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')
    #url = 'https://coda.eumetsat.int'
    api = SentinelAPI(user, password, url)

    #footprint = 'POLYGON ((3.632678257283426 44.218520813901634, 5.593941712716586 43.91153565143112,5.122306465193042 42.40131912218008, 3.208397257788463 42.706154058381806,3.632678257283426 44.218520813901634))'
    #date = ('20191001','NOW')
    products = api.query(footprint,
                         date = (datestart,dateend),
                         platformname = 'Sentinel-3',
                         area_relation='Intersects', 
                         producttype='OL_2_WFR___',
                         #timeliness='Non Time Critical',
                         instrumentshortname = 'OLCI',
                         onlinequalitycheck = 'PASSED',
                         productlevel = 'L2')
    # convert to Pandas DataFrame
    products_df = api.to_dataframe(products)
    # sort and limit to first 5 sorted products
    products_df_sorted = products_df.sort_values(['salinewatercoverpercentage', 'ingestiondate'], ascending=[False, True])
    products_df_sorted = products_df_sorted.head(image_number)
    # download sorted and reduced products
    api.download_all(products_df_sorted.index)
    #zip_file_path = "/home/courtois/jupyter-sentinel"
    file_list = os.listdir(zip_file_path+"/")
    # select the directory where the files are and where you want to unzip it
    abs_path = []
    # list for the .zip file to unzip
    c=0
    for a in file_list:
        if a.endswith(".zip"):  # select only the .zip files in the folder
            x = zip_file_path+'/'+a
            c += 1
            print x
            abs_path.append(x)  # put the .zip file in the list
    for f in abs_path:
        zip=zipfile.ZipFile(f)
        zip.extractall(zip_file_path)  # unzip the file in the zip_file_path
        os.remove(f)   # delet the .zip file
        
    print("File in :" + zip_file_path)
    print("DONE")

Exemple d'utilisation de la fonction

In [3]:
user = 'jonathanc'
password = 'Tutorat2019'
url = 'https://coda.eumetsat.int'
footprint = 'POLYGON ((3.632678257283426 44.218520813901634, 5.593941712716586 43.91153565143112,5.122306465193042 42.40131912218008, 3.208397257788463 42.706154058381806,3.632678257283426 44.218520813901634))'
datestart = ('20191001')
dateend = ('NOW')
zip_file_path = "/home/courtois/jupyter-sentinel"
image_number = 10
Download_olci_data(user,password,url,footprint,datestart,dateend,zip_file_path,image_number)

Querying products: 100%|██████████| 127/127 [00:01<00:00, 107.12 products/s]
Downloading: 100%|██████████| 305M/305M [00:24<00:00, 12.6MB/s] 
Downloading: 100%|██████████| 349M/349M [00:27<00:00, 12.8MB/s] 
Downloading: 100%|██████████| 296M/296M [00:23<00:00, 12.8MB/s] 
Downloading: 100%|██████████| 281M/281M [00:21<00:00, 12.8MB/s] 
Downloading: 100%|██████████| 282M/282M [00:22<00:00, 12.8MB/s] 
Downloading: 100%|██████████| 270M/270M [00:21<00:00, 12.8MB/s] 
Downloading: 100%|██████████| 257M/257M [00:20<00:00, 12.8MB/s] 
Downloading: 100%|██████████| 249M/249M [00:19<00:00, 12.9MB/s] 
Downloading: 100%|██████████| 234M/234M [00:18<00:00, 12.8MB/s] 
Downloading: 100%|██████████| 231M/231M [00:17<00:00, 12.9MB/s] 


/home/courtois/jupyter-sentinel/S3B_OL_2_WFR____20191121T103322_20191121T103622_20191204T200058_0179_032_222_2160_MAR_O_NT_002.zip
/home/courtois/jupyter-sentinel/S3B_OL_2_WFR____20191006T102550_20191006T102850_20191007T190005_0179_030_336_2160_MAR_O_NT_002.zip
/home/courtois/jupyter-sentinel/S3B_OL_2_WFR____20191026T100707_20191026T101007_20191027T191130_0179_031_236_2160_MAR_O_NT_002.zip
/home/courtois/jupyter-sentinel/S3B_OL_2_WFR____20191121T103322_20191121T103622_20191121T123442_0179_032_222_2160_MAR_O_NR_002.zip
/home/courtois/jupyter-sentinel/S3B_OL_2_WFR____20191010T102205_20191010T102505_20191011T191956_0180_031_008_2160_MAR_O_NT_002.zip
/home/courtois/jupyter-sentinel/S3A_OL_2_WFR____20191004T101649_20191004T101949_20191005T174633_0179_050_065_2160_MAR_O_NT_002.zip
/home/courtois/jupyter-sentinel/S3B_OL_2_WFR____20191203T102209_20191203T102509_20191203T122408_0179_033_008_2160_MAR_O_NR_002.zip
/home/courtois/jupyter-sentinel/S3B_OL_2_WFR____20191008T093328_20191008T093628_201